In [137]:
import requests
import time
import re
import polars as pl
from bs4 import BeautifulSoup
from urllib.parse import urlencode, urlunparse
from thefuzz import process

In [165]:
def get_formated_url(path):
    # Define the base components of the URL
    scheme = 'https'
    netloc = 'transfermarkt-api.vercel.app'
    params = ''
    query = urlencode({'page_number': 1})  # Converts parameters to URL-encoded string
    fragment = ''
    # Construct the URL
    url = urlunparse((scheme, netloc, path, params, query, fragment))

    return url

def get_transfertmarkt_team_id(team_name, team_id):
    path = f"/clubs/search/{team_name}"  # Spaces will be encoded automatically
    url = get_formated_url(path)

    # Set the headers
    headers = {
        'accept': 'application/json'
    }

    # Make the GET request
    response = requests.get(url, headers=headers)

    # Check if the request was successful
    if response.status_code == 200:
        # Convert the response to JSON
        if response.json():
            data = response.json()
            return {'TeamName': team_name, 'TeamId': team_id, 'results': data}
        else:
            return {'TeamName': team_name, 'TeamId': team_id, 'results': None}
    else:
        print(f"Failed to retrieve data: {response.status_code}")
        return {'TeamName': team_name, 'TeamId': team_id, 'results': None}


def get_players_from_team(t_team_id):
    # Define the URL and parameters
    path = f"/clubs/{t_team_id}/players"
    url = get_formated_url(path)

    # Set the headers
    headers = {
        'accept': 'application/json'
    }

    # Make the GET request
    response = requests.get(url, headers=headers)

    # Check if the request was successful
    if response.status_code == 200:
        # Convert the response to JSON
        return response.json()
    else:
        print(f"Failed to retrieve data: {response.status_code}")
        return None

def scrap_team_transfertmarkt(team_id):
    headers = {
        'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/47.0.2526.106 Safari/537.36'
    }

    try:
        page = f"https://www.transfermarkt.fr/elfenbeinkuste/kader/verein/{team_id}"
        response = requests.get(page, headers=headers)
        response.raise_for_status()  # Raise an error for bad status codes
        pageSoup = BeautifulSoup(response.content, 'html.parser')

        players_info = extract_players_information(pageSoup)
        
        return players_info

    except requests.RequestException as e:
        print(f"Request error: {e}")
        return None
    except ValueError as e:
        print(f"Request error: {e}")
        return None

def extract_players_information(pageSoup):
    # Initialize an empty list to store player information
    players_info = []

    # Iterate through each player entry in the HTML
    for row in pageSoup.find_all('tr', {'class': ['even', 'odd']}):
        player_link = row.find('a', href=re.compile(r'/profil/spieler/'))
        player_name = player_link.text.strip()
        position = row.find_all('td')[4].text.strip()  # Adjust index based on HTML structure

        players_info.append({
            'PlayerName': player_name,
            'PositionName': position
        })

    return players_info

## Read player table

In [172]:
player_df = pl.read_csv('raw\Player.csv', separator=';')
player_team_df = pl.read_csv('raw\PlayerTeam.csv', separator=';')

player_to_process = player_df \
    .select(['PlayerId', 'PlayerName', 'Position']) \
    .filter(pl.col("Position").is_in(['SUB', 'RES'])) \
    .join(player_team_df, on='PlayerId', how='left') \
    .drop('Position', 'Link') 

teams = player_to_process[['Team', 'TeamId']].unique()

## Get teams of player with no position

In [173]:
team_info = []
nb_teams_done = 0 
for row in teams.iter_rows():
    team_name = row[0]
    team_id = row[1]
    res = get_transfertmarkt_team_id(team_name, team_id)['results']['results']
    if res:
        team_transfertmarkt_id = res[0]['id']
        team_info.append([team_name, team_id, team_transfertmarkt_id])
    else:
        team_info.append([team_name, team_id, None])

    nb_teams_done = nb_teams_done + 1
    if nb_teams_done %10 == 0:
        print(f"Number of teams done : {nb_teams_done}")

Number of teams done : 10
Number of teams done : 20
Number of teams done : 30
Number of teams done : 40
Number of teams done : 50
Number of teams done : 60
Number of teams done : 70
Number of teams done : 80
Number of teams done : 90
Number of teams done : 100
Number of teams done : 110
Number of teams done : 120
Number of teams done : 130
Number of teams done : 140
Number of teams done : 150
Number of teams done : 160
Number of teams done : 170
Number of teams done : 180
Number of teams done : 190
Number of teams done : 200
Number of teams done : 210
Number of teams done : 220
Number of teams done : 230
Number of teams done : 240
Number of teams done : 250
Number of teams done : 260
Number of teams done : 270


In [174]:
team_info = pl.DataFrame(team_info, schema = ['TeamName', 'TeamId', 'TransfertmarktId'])
team_info.head()

TeamName,TeamId,TransfertmarktId
str,i64,str
"""Swansea City""",17,"""2288"""
"""Chelsea""",203,"""631"""
"""Schalke 04""",162,"""33"""
"""Aston Villa""",198,"""405"""
"""Samsunspor""",261,"""152"""


In [175]:
print(team_info.filter(pl.col("TransfertmarktId").is_null()))

shape: (7, 3)
┌────────────────────┬────────┬──────────────────┐
│ TeamName           ┆ TeamId ┆ TransfertmarktId │
│ ---                ┆ ---    ┆ ---              │
│ str                ┆ i64    ┆ str              │
╞════════════════════╪════════╪══════════════════╡
│ Bari 1908          ┆ 223    ┆ null             │
│ VfL Bochum 1848    ┆ 137    ┆ null             │
│ Gaziantep F.K.     ┆ 264    ┆ null             │
│ Sporting Charleroi ┆ 148    ┆ null             │
│ Sporting Braga     ┆ 238    ┆ null             │
│ Quevilly Rouen     ┆ 38     ┆ null             │
│ Villarreal II      ┆ 80     ┆ null             │
└────────────────────┴────────┴──────────────────┘


### Manual correction

In [176]:
team_info = team_info.with_columns(
    pl.when(pl.col("TeamName") == 'Gaziantep F.K.').then(2832) \
    .when(pl.col("TeamName") == 'Villarreal II').then(11972) \
    .when(pl.col("TeamName") == 'Bari 1908').then(332) \
    .when(pl.col("TeamName") == 'VfL Bochum 1848').then(80) \
    .when(pl.col("TeamName") == 'Quevilly Rouen').then(7124) \
    .when(pl.col("TeamName") == 'Sporting Braga').then(1075) \
    .when(pl.col("TeamName") == 'Sporting Charleroi').then(172) \
    .otherwise(pl.col("TransfertmarktId"))
    .alias("TransfertmarktId")
)

team_info.filter(pl.col("TransfertmarktId").is_in(['2832', '11972', '332', '80', '7124', '1075', '172']))

TeamName,TeamId,TransfertmarktId
str,i64,str
"""Bari 1908""",223,"""332"""
"""VfL Bochum 184…",137,"""80"""
"""Gaziantep F.K.…",264,"""2832"""
"""Sporting Charl…",148,"""172"""
"""Sporting Braga…",238,"""1075"""
"""Quevilly Rouen…",38,"""7124"""
"""Villarreal II""",80,"""11972"""


## Extract correct players from team

In [ ]:
players_list = []

# Initialize the index
index = 0

# Convert team_info to a list of rows if it's not already in that format
team_rows = list(team_info.iter_rows())

# Start the while loop
while index < len(team_rows):
    row = team_rows[index]
    t_team_id = row[2]
    players_info = None

    # Attempt to get players info from the team
    while players_info is None:
        players_info = scrap_team_transfertmarkt(t_team_id)

        # Check if players_info is not None before processing
        if players_info:
            players_list.append(pl.DataFrame(players_info))
        else:
            time.sleep(20) # seconds

    # Increment the index to move to the next team
    index += 1
    if index %10 == 0:
        print(f"Number of teams done : {index}")
    

In [296]:
full_players_df = pl.concat(players_list)
full_players_df.shape

(7427, 2)

In [297]:
players_without_positions = player_to_process['PlayerName'].to_list()

In [289]:
print(f"Number of players with positions : {full_players_df.shape[0]}")
print(f"Number of players without positions : {len(players_without_positions)}")

Number of players with positions : 7427
Number of players without positions : 4853


In [298]:
# Create a dictionary to hold the best matches for each player to process
matches = []

while len(players_without_positions) > 0:
    players_with_positions = full_players_df['PlayerName'].to_list()
    # get the first name on that list and then delete it from this list
    player = players_without_positions.pop()
    # find the match
    player_found = process.extractOne(player, players_with_positions)[0]
    # extract the first position
    first_position = full_players_df.filter(pl.col("PlayerName") == player_found)["PositionName"][0]
    # filter out every rows having this name
    full_players_df = full_players_df.filter(~(pl.col("PlayerName") == player_found))
    # keep results
    matches.append([player, first_position])

In [299]:
matche_df = pl.DataFrame(matches, schema=['PlayerName', 'NewPosition'])
matche_df.shape

(4853, 2)

In [300]:
player_name_df = pl.read_csv('silver\PlayerName.csv')

matche_df = matche_df.join(player_name_df, on='PlayerName', how='left')
matche_df.head()

PlayerName,NewPosition,PlayerId
str,str,i64
"""Eray Korkmaz""","""Arrière droit""",7800
"""Rayane Aabid""","""Milieu central…",7694
"""Renat Oleq Oğl…","""Avant-centre""",7695
"""Mehdi Boudjema…","""Milieu central…",7697
"""Kerim Alıcı""","""Arrière droit""",7696


In [303]:
# Perform a left join to match and merge the new positions with the existing ones
df_joined = player_df.join(matche_df, on='PlayerId', how='left')

In [306]:
# Update the 'Position' only if 'NewPosition' is not null; otherwise, keep the original 'Position'
df_updated = df_joined.with_columns(
    pl.when(pl.col("NewPosition").is_null())
    .then(pl.col("Position"))
    .otherwise(pl.col("NewPosition"))
    .alias("UpdatedPosition")
)

In [307]:
df_final = df_updated.drop(["Position", "NewPosition"]).rename({"UpdatedPosition": "Position"})

## Save data

In [310]:
df_final.write_csv('raw/Player.csv', separator=";")